In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

# 簡單的Agent demo
- https://www.langchain.com.cn/docs/tutorials/agents/
- https://app.tavily.com/home?code=y8T47whUH7HY5akkEpvHh3paUISRO50TQhUVTb4fn_UUS&state=eyJyZXR1cm5UbyI6Ii9ob21lIn0

In [2]:
# Import relevant functionality
from langchain_ollama import OllamaLLM, ChatOllama

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatOllama(model='llama3.2:3b')
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-01-05T07:37:14.8954348Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': {'query': 'San Francisco resident information'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 22126938200, 'load_duration': 19810185100, 'prompt_eval_count': 195, 'prompt_eval_duration': 597000000, 'eval_count': 24, 'eval_duration': 1716000000}, id='run-c1fa2528-cd2b-47b6-a405-66adf01683ec-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco resident information'}, 'id': '9ad3e3ea-1877-4cf5-9a13-8340f1db1488', 'type': 'tool_call'}], usage_metadata={'input_tokens': 195, 'output_tokens': 24, 'total_tokens': 219})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.sfgov.org/residents", "content": "San francisco is a unique city that offe

# 以下為Demo的各部分拆解

# Tools
我們首先需要創建我們想要使用的工具。我們主要選擇的工具將是Tavily - 一個搜尋引擎。 LangChain內建了一個工具，可以輕鬆使用Tavily搜尋引擎作為工具

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Taiwan?")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

# LLM with tools

In [ ]:
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2:3b')

In [ ]:
#您可以透過傳入訊息清單來呼叫語言模型。
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

In [ ]:
# 綁定tools的方法
model_with_tools = model.bind_tools(tools)

In [ ]:
# 觀察tool_calls
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
# 觀察tool_calls
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")
# 我們可以看到現在沒有文字內容，但有一個工具呼叫！它希望我們呼叫Tavily Search 工具。
# 這還沒有調用那個工具- 它只是告訴我們去調用。為了實際調用它，我們需要創建我們的代理。

# Create Agent


In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [ ]:
# 請注意，代理將在交互結束時返回最終狀態（包括任何輸入，我們稍後將看到如何僅獲取輸出）。

# 當不需要工具時 (這邊發生了意外，llama3在我打招呼的時候就會去找最近的新聞跟我分享XD)
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

In [ ]:
# 當需要工具時
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Taipei?")]}
)
response["messages"]

In [ ]:
# AI 的第一個message可以查看呼叫的tool
response["messages"][1].tool_calls

# Streaming

In [ ]:
# 每次message都即時顯示出來
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

## astream_events (沒讀懂)
除了流式返回訊息，
流式返回令牌也是有用的。
我們可以使用.astream_events方法來實現這一點

In [ ]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="why sky is blue?")]}, version="v1"
):
    kind = event["event"]
    # print(kind)
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

# Checkpoint Memory

In [ ]:
# 如前所述，該代理是無狀態的。這意味著它不記得先前的互動。
# 為了給它添加內存，我們需要傳入一個檢查點。
# 當傳入檢查點時，我們還必須在呼叫代理時傳入thread_id（以便它知道從哪個執行緒/對話恢復）
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()


In [ ]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Hello, I am Jason. How are you?")]}, config
):
    print(chunk)
    print("----")

In [ ]:
# 注意，我換了一個thread_id，agent就不認識我了!
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Whats my name?")]}, config
):
    print(chunk)
    print("----")